In [1]:
from pyspark.sql import SparkSession, Row, SQLContext
from pyspark.sql.functions import udf, col, date_format
import pyspark.sql.types as T
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, IDF, RegexTokenizer, StopWordsRemover
from pyspark.ml.clustering import LDA, LocalLDAModel
from pyspark.ml import Pipeline
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_style("whitegrid")
import pandas as pd
from wordcloud import WordCloud
import re
import nltk

In [2]:
spark = SparkSession.builder.appName('NLP').getOrCreate()

## Preprocess

In [3]:
df = spark.read.csv("data/merged_df_final.csv", header=True)

In [4]:
df = df[['province_state', 'date', 'confirmed_state', 'deaths_state', 'governor_tweet']]
df = df.drop_duplicates(subset=['governor_tweet', 'province_state'])
df = df.dropna()

In [5]:
df.show()

+--------------+----------+---------------+------------+--------------------------+
|province_state|      date|confirmed_state|deaths_state|            governor_tweet|
+--------------+----------+---------------+------------+--------------------------+
|       Florida|2020/08/31|       623471.0|     11187.0|      (8/8): ED visits ...|
|      Illinois|2020/04/26|        43903.0|      1933.0|      4/26 COVID-19 Dai...|
|      Delaware|2020/06/03|         9712.0|       375.0|      As businesses reo...|
|  North Dakota|2021/01/15|        95599.0|      1395.0|      At 3:00 p.m. we w...|
|      Missouri|2020/06/01|        13724.0|       776.0|      Before COVID-19, ...|
|        Oregon|2020/04/30|         2510.0|       103.0|COVID-19에 관한 이 자료...|
|      Illinois|2020/06/25|       139434.0|      6810.0|      Como resultado, h...|
|     Louisiana|2020/08/26|       144960.0|      4851.0|      Despite progress ...|
|        Oregon|2020/11/14|        56018.0|       759.0|      During the Two-We...

In [7]:
texts=df.rdd.map(lambda x: x['governor_tweet'])
tweets=texts.zipWithIndex()
data = spark.createDataFrame(tweets, ["tweets",'index'])

removePunct = udf(
    lambda s: re.sub(r'[^a-zA-Z0-9]|[0-9]', r' ', s).strip().lower(), T.StringType())

# normalize the post content (remove html tags, punctuation and lower case..)
data_norm = data.withColumn("text", removePunct(data.tweets))

In [11]:
tokenizer = RegexTokenizer(inputCol="text", outputCol="words",
                           gaps=True, pattern=r'\s+', minTokenLength=2)
df_tokens = tokenizer.transform(data_norm)

nltk.download('stopwords')#must be downloaded to run
stop_words = stopwords.words("english") + ["which", "amp", "who", "whose", "xa", "de", "http"]
removeStop=udf(lambda word: [x for x in word if x not in stop_words])
df_tokens=df_tokens.withColumn('noStopWords',removeStop(df_tokens['words']))

label_udf = udf(lambda x: x, T.ArrayType(T.StringType())) 
df_tokens=df_tokens.withColumn('final_words',label_udf(df_tokens.noStopWords))

[nltk_data] Downloading package stopwords to /Users/miao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
df_tokens.printSchema()

root
 |-- tweets: string (nullable = true)
 |-- index: long (nullable = true)
 |-- text: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- noStopWords: string (nullable = true)
 |-- final_words: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [12]:
df_tokens[['tweets', 'index', 'final_words']].show()

+--------------------------+-----+--------------------+
|                    tweets|index|         final_words|
+--------------------------+-----+--------------------+
|      (8/8): ED visits ...|    0|[ed, visits, covi...|
|      4/26 COVID-19 Dai...|    1|[covid, daily, br...|
|      As businesses reo...|    2|[businesses, reop...|
|      At 3:00 p.m. we w...|    3|[host, covid, pre...|
|      Before COVID-19, ...|    4|[covid, one, prio...|
|COVID-19에 관한 이 자료...|    5|[covid, https, co...|
|      Como resultado, h...|    6|[como, resultado,...|
|      Despite progress ...|    7|[despite, progres...|
|      During the Two-We...|    8|[two, week, freez...|
|      Georgia continues...|    9|[georgia, continu...|
|      Great job Galvest...|   10|[great, job, galv...|
|      I got my first CO...|   11|[got, first, covi...|
|      I will be testify...|   12|[testifying, subc...|
|      Ill be joined by ...|   13|[ill, joined, cvs...|
|      It’s incumbent up...|   14|[incumbent, upon,...

## TF-IDF

In [13]:
%%time
# TF
#VocabSize of 20000 words and words with frequencies above 10
cv = CountVectorizer(inputCol="final_words", outputCol="raw_features",vocabSize=20000, minDF=10.0)
cvmodel = cv.fit(df_tokens)

result_cv = cvmodel.transform(df_tokens)

CPU times: user 34.3 ms, sys: 13.7 ms, total: 47.9 ms
Wall time: 14.5 s


In [14]:
%%time
# IDF
idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)
result_tfidf = idfModel.transform(result_cv)

CPU times: user 25.8 ms, sys: 11.1 ms, total: 36.8 ms
Wall time: 7.2 s


In [15]:
%%time
#split 80% train set and 20% test set
df_training, df_testing = result_tfidf.randomSplit([0.8, 0.2], 1)
print('Training and testing documents: ', df_training.count(), df_testing.count())

num_topics=10
max_iterations=50
lda = LDA(k=num_topics, maxIter=max_iterations)
ldaModel = lda.fit(result_tfidf )

Training and testing documents:  8214 2098
CPU times: user 945 ms, sys: 446 ms, total: 1.39 s
Wall time: 1min 18s


## Result

In [17]:
topics = ldaModel.describeTopics(maxTermsPerTopic=5)
vocabArray = cvmodel.vocabulary
numTopics=10

ListOfIndexToWords = udf(lambda wl: list([vocabArray[w] for w in wl]))
FormatNumbers = udf(lambda nl: ["{:1.4f}".format(x) for x in nl])

toptopics = topics.select((topics.topic + 1).alias('topic'),
                          ListOfIndexToWords(topics.termIndices).alias('words'),
                          FormatNumbers(topics.termWeights).alias('weights'))
toptopics.show(truncate=False, n=numTopics)
print('Topics:', numTopics, 'Vocabulary:', len(vocabArray))

+-----+---------------------------------------------+----------------------------------------+
|topic|words                                        |weights                                 |
+-----+---------------------------------------------+----------------------------------------+
|1    |[testing, get, test, find, sites]            |[0.0242, 0.0126, 0.0108, 0.0096, 0.0095]|
|2    |[spread, mask, stay, keep, wear]             |[0.0145, 0.0116, 0.0112, 0.0101, 0.0097]|
|3    |[statewide, deaths, cases, additional, new]  |[0.0294, 0.0276, 0.0273, 0.0248, 0.0225]|
|4    |[watch, live, briefing, pm, director]        |[0.0260, 0.0206, 0.0203, 0.0190, 0.0148]|
|5    |[patients, care, force, hospitals, task]     |[0.0189, 0.0153, 0.0112, 0.0112, 0.0110]|
|6    |[businesses, help, small, support, business] |[0.0135, 0.0108, 0.0097, 0.0084, 0.0079]|
|7    |[vaccine, doses, care, state, first]         |[0.0124, 0.0087, 0.0083, 0.0072, 0.0068]|
|8    |[information, call, response, provides, liv

## Evaluate

In [16]:
lpt, lp = ldaModel.logPerplexity(df_testing), ldaModel.logPerplexity(df_training)
print("Perplexity on testing and training data: " + str(lp) + ',' + str(lpt))

Perplexity on testing and training data: 6.69577013712447,6.941993599041921


In [20]:
llt, ll = ldaModel.logLikelihood(df_testing), ldaModel.logLikelihood(df_training)
print("Likelihood on testing and training data: " + str(llt) + ',' + str(ll))

Likelihood on testing and training data: -986748.4898593953,-3699021.042064962


In [ ]:
#Reference: https://github.com/alejandronotario/LDA-Topic-Modeling/